In [1]:
import os
import pickle
import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB

import import_ipynb
from bohb_worker_1to3_6 import KerasWorker as worker

parser = argparse.ArgumentParser(description='bohb_server')
parser.add_argument('--min_budget',   type=float, help='Minimum number of epochs for training.',    default=50)
parser.add_argument('--max_budget',   type=float, help='Maximum number of epochs for training.',    default=200)
parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=100)
parser.add_argument('--worker', help='Flag to turn this into a worker process', action='store_true')
parser.add_argument('--run_id', type=str, help='A unique run id for this optimization run. An easy option is to use the job id of the clusters scheduler.')
parser.add_argument('--nic_name',type=str, help='Which network interface to use for communication.', default='lo')
parser.add_argument('--shared_directory',type=str, help='A directory that is accessible for all processes, e.g. a NFS share.', default='../BOHB_LAB_5CNN/result/6')

args=parser.parse_args(args=[])
    
# Every process has to lookup the hostname
host = hpns.nic_name_to_host(args.nic_name)


if args.worker:
    import time
    time.sleep(5)   # short artificial delay to make sure the nameserver is already running
    w = worker(run_id=args.run_id, host=host, timeout=120)
    w.load_nameserver_credentials(working_directory=args.shared_directory)
    w.run(background=False)
    exit(0)

# This example shows how to log live results. This is most useful
# for really long runs, where intermediate results could already be
# interesting. The core.result submodule contains the functionality to
# read the two generated files (results.json and configs.json) and
# create a Result object.
result_logger = hpres.json_result_logger(directory=args.shared_directory, overwrite=False)

# Start a nameserver:
NS = hpns.NameServer(run_id=args.run_id, host=host, port=0, working_directory=args.shared_directory)
ns_host, ns_port = NS.start()
# NS = hpns.NameServer(run_id=args.run_id, host='127.0.0.1', port=None, working_directory=args.shared_directory)
# ns_host, ns_port = NS.start()

# Start local worker
w = worker(run_id=args.run_id, host=host, nameserver=ns_host, nameserver_port=ns_port, timeout=120)
w.run(background=True)

# Run an optimizer
bohb = BOHB(  configspace = worker.get_configspace(),
                      run_id = args.run_id,
                      host=host,
                      nameserver=ns_host,
                      nameserver_port=ns_port,
                      result_logger=result_logger,
                      min_budget=args.min_budget, max_budget=args.max_budget,
               )
res = bohb.run(n_iterations=args.n_iterations)

# store results
with open(os.path.join(args.shared_directory, 'results.pkl'), 'wb') as fh:
    pickle.dump(res, fh)

# shutdown
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

importing Jupyter notebook from bohb_worker_1to3_6.ipynb
(11, 504960, 2)
(11, 101280, 2)


DEBUG:hpbandster:wait_for_workers trying to get the condition
INFO:hpbandster:DISPATCHER: started the 'discover_worker' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7fee57656c50; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:43841>
INFO:hpbandster:DISPATCHER: started the 'job_runner' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: No dispatcher found. Waiting for one to initiate contact.
INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: start listening for jobs
INFO:hpbandster:DISPATCHER: Pyro daemon running on 127.0.0.1:43483
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 0 currently in the pool.
INFO:hpbandster:DISPATCHER: discovered new worker, hpbandster.run_None.worker.74c293875977.63425140674247952192
DEBUG:hpbandster:HBMASTER: number of workers changed to 1
DEBUG:hpbandster:Enough workers to start this run!
DEBUG:hp

(11572, 480, 2)
(2321, 480, 2)


DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: done with job (0, 0, 0), trying to register it.
DEBUG:hpbandster:DISPATCHER: job (0, 0, 0) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
INFO:hpbandster.

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: done with job (0, 0, 2), trying to register it.
INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: registered result for job (0, 0, 2) with dispatcher
DEBUG:hpbandster:DISPATCHER: job (0, 0, 2) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (0, 0, 2) on hpbandster.run_None.worker.74c293875977.63425140674247952192 finished
DEBUG:hpbandster:job_id: (0, 0, 2)
kwargs: {'config': {'dropout_rate': 0.4102344304402984, 'lr': 2.6044351757949174e-05, 'lstm_units': 242, 'num_conv_layers': 5, 'num_fc_l

...

DEBUG:hpbandster:DISPATCHER: starting job (97, 0, 0) on hpbandster.run_None.worker.74c293875977.63425140674247952192
DEBUG:hpbandster:DISPATCHER: job (97, 0, 0) dispatched on hpbandster.run_None.worker.74c293875977.63425140674247952192
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: start processing job (97, 0, 0)
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: kwargs: {'config': {'dropout_rate': 0.17102357554328704, 'lr': 2.448379012390306e-05, 'lstm_units': 172, 'num_conv_layers': 3, 'num_fc_layers': 3, 'num_fc_units_1': 9, 'num_filters_1': 35, 'optimizer': 'Adam', 'num_fc_units_2': 13, 'num_fc_units_3': 22, 'num_filters_2': 7, 'num_filters_3': 129}, 'budget': 200.0, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 cur

DEBUG:hpbandster:DISPATCHER: trying to notify the job_runner thread.
DEBUG:hpbandster:HBMASTER: job (98, 0, 0) submitted to dispatcher
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
DEBUG:hpbandster:DISPATCHER: starting job (98, 0, 0) on hpbandster.run_None.worker.74c293875977.63425140674247952192
DEBUG:hpbandster:DISPATCHER: job (98, 0, 0) dispatched on hpbandster.run_None.worker.74c293875977.63425140674247952192
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: start processing job (98, 0, 0)
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: kwargs: {'config': {'dropout_rate': 0.446063608417819, 'lr': 0.002432714070022329, 'lstm_units': 129, 'num_conv_layers': 1, 'num_fc_layers': 2, 'num_fc_units_1': 24, 'num_filters_1': 53, 'op

INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: start processing job (98, 0, 2)
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: kwargs: {'config': {'dropout_rate': 0.7681607925559998, 'lr': 0.02397349881555149, 'lstm_units': 194, 'num_conv_layers': 3, 'num_fc_layers': 2, 'num_fc_units_1': 223, 'num_filters_1': 74, 'optimizer': 'SGD', 'num_fc_units_2': 45, 'num_filters_2': 84, 'num_filters_3': 188, 'sgd_momentum': 0.3291327327521539}, 'budget': 66.66666666666666, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.


INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: start processing job (99, 0, 0)
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: kwargs: {'config': {'dropout_rate': 0.18846583335668576, 'lr': 0.0007716642324238174, 'lstm_units': 176, 'num_conv_layers': 2, 'num_fc_layers': 1, 'num_fc_units_1': 51, 'num_filters_1': 37, 'optimizer': 'Adam', 'num_filters_2': 23}, 'budget': 200.0, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977.63425:WORKER: done with job (99, 0, 1), trying to register it.
INFO:hpbandster.run_None.worker.74c293875977.63425:WORKER: registered result for job (99, 0, 1) with dispatcher
DEBUG:hpbandster:DISPATCHER: job (99, 0, 1) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (99, 0, 1) on hpbandster.run_None.worker.74c293875977.63425140674247952192 finished
DEBUG:hpbandster:job_id: (99, 0, 1)
kwargs: {'config': {'dropout_rate': 0.544915333036836, 'lr': 0.0017022037785554937, 'lstm_units': 98, 'num_conv_layers': 1, 'num_fc

In [2]:
# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()

print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in res.get_all_runs()])/args.max_budget))

Best found configuration: {'dropout_rate': 0.16519178106690297, 'lr': 0.0016556065892678969, 'lstm_units': 159, 'num_conv_layers': 2, 'num_fc_layers': 3, 'num_fc_units_1': 8, 'num_filters_1': 48, 'optimizer': 'SGD', 'num_fc_units_2': 241, 'num_fc_units_3': 13, 'num_filters_2': 128, 'sgd_momentum': 0.9102646030327244}
A total of 250 unique configurations where sampled.
A total of 300 runs where executed.
Total budget corresponds to 200.0 full function evaluations.
